In [38]:
import openai
import shutil
import json
import os
from dotenv import load_dotenv

load_dotenv('.env.local')
openai.api_key = os.getenv("OPENAI_API_KEY")

In [21]:
def generate_prompt(topic):
    prompt = f'Topic: {topic}\n' + '''
    Provide a JSON object containing the topic, a list of 1-12 prerequisite topics, and a list of 1-12 further readings related to AI, ML, and DL. 
    Ensure that the prerequisites and further readings are specifically relevant to the given, rather than broad topics like calculus or statistics. 
    Use a similar format to the example provided below:

    Example:
    {
        "generative_adversarial_network": {
            "title": "Generative Adversarial Network",
            "prerequisites": ["expectation_maximization_algorithm", "probability_distributions", "convolutional_neural_networks", "backpropagation", "stochastic_gradient_descent", "loss_functions", "optimization_algorithms", "deep_learning_frameworks", "regularization_techniques", "unsupervised_learning"],
            "further_readings": ["conditional_gans", "cycle_gans", "stylegan_and_stylegan2", "wasserstein_gans", "domain_adaptation", "image_to_image_translation", "semi_supervised_learning", "adversarial_training", "adversarial_attacks_and_defenses", "transfer_learning"]
        }
    }
    Next, write a detailed wiki page about the given topic in Markdown format, from the perspective of a world-renowned AI and ML expert. 
    Ensure that this wiki page is explicitly in code format. Do not include a "Contents" section nor a "Further Readings" section.
    Use a neutral, unbiased tone without exclamation marks, and write in the style of a melancholic older person. 
    Follow Markdown syntax for headings and formatting, and use LaTeX for equations, with inline equations in pairs of $ and multiline equations in $$. 
    Ensure the entire output is less than 8192 tokens long and does not include an extra line at the end of the Markdown.
    '''
    return prompt

In [4]:
topic = 'gan'
prompt = generate_prompt(topic)

completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": prompt}
    ],
    temperature=0.7,
)

In [ ]:
finish_reason = completion.choices[0]['finish_reason']
message = completion.choices[0].message.content

print(finish_reason)

json_start_pos = message.find('"')
json_end_pos = message.find('}')
json_string = message[json_start_pos:json_end_pos + 1]
if json_string.count('{') == 1 and json_string.count('}') == 1:
    json_string = '{' + json_string.strip() + '}'
json_object = json.loads(json_string)

with open('wiki-connections.json', 'r') as file:
    existing_data = json.load(file)

existing_data.update(json_object)
with open('wiki-connections.json', 'w') as file:
    json.dump(existing_data, file, indent=4)


In [45]:
markdown_start_pos = message.find('#', json_end_pos)
markdown_content = message[markdown_start_pos:].strip() + '\n'

md_filename = topic + '.md'
with open(md_filename, 'w') as file:
    file.write(markdown_content)

destination_folder = 'data'
shutil.move(md_filename, destination_folder)

'data/blah.md'

In [48]:
js_string = f'''
import React from 'react';
import path from 'path';
import fs from 'fs';
import PageContent from '@/components/PageContent/PageContent';

const filename = '{md_filename}';

export default function MarkdownPage({{ markdownContent }}) {{
  return <PageContent content={{markdownContent}} filename={{filename}} />;
}}

export async function getStaticProps() {{
  const filePath = path.join(process.cwd(), 'data', filename);
  const markdownContent = fs.readFileSync(filePath, 'utf8');
  return {{
    props: {{
      markdownContent,
    }},
  }};
}}
'''

js_filename = topic + '.js'
with open(js_filename, 'w') as file:
    file.write(js_string)

destination_folder = 'pages'
shutil.move(js_filename, destination_folder)

'pages/blah.js'